In [1]:
from __future__ import print_function, division

In [2]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys
import time
import shutil

import urllib

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')
if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [3]:
import hosts
import decals

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy import table
from astropy.table import Table
from astropy.io import fits

import tqdm
from astropy.utils import data

In [4]:
%matplotlib inline
from matplotlib import style, pyplot as plt

plt.style.use('seaborn-deep')
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['figure.figsize'] = (14, 8)

# Load the bricks and SAGA hosts

In [25]:
bricks = Table.read('decals_dr4/survey-bricks.fits.gz')
bricksdr3 = Table.read('decals_dr3/survey-bricks-dr3.fits.gz')
bricksdr4 = Table.read('decals_dr4/survey-bricks-dr4.fits.gz')

In [26]:
paper1nsas_comp = [166313,147100,165536,61945,132339,149781,33446,150887]
paper1nsas_incomp = [161174,85746,145729,140594,126115,13927,137625,129237]
paper1nsas = paper1nsas_comp + paper1nsas_incomp

In [27]:
hostobjs = hosts.get_saga_hosts_from_google()
paperhosts = [h for h in hostobjs if h.nsaid in paper1nsas]
assert len(paperhosts) == len(paper1nsas)

paperhostscs = SkyCoord([h.coords for h in paperhosts])
paperhostsbyname = {h.name:h for h in paperhosts}

Using cached version of google hosts list from file "hosts_dl.pkl2"


In [28]:
host_bricks_3 = decals.find_host_bricks(paperhosts, bricksdr3, bricks)
host_bricks_4 = decals.find_host_bricks(paperhosts, bricksdr4, bricks)

host_bricks_dct = {3:host_bricks_3, 4:host_bricks_4}

# Build brick catalogs for every host

In [20]:
for hostobj in paperhosts:
    hbricks = host_bricks_3[host_bricks_3['closest_host_name'] == hostobj.name]
    catfn = 'decals_dr3/'

<hosts.NSAHost object w/ name 'Odyssey' AKA: ['NGC6181', 'NSA147100']> 31
<hosts.NSAHost object w/ name 'Alice' AKA: ['NGC4030', 'NSA140594']> 56
<hosts.NSAHost object w/ name 'Gilgamesh' AKA: ['NGC5962', 'NSA166313']> 38
<hosts.NSAHost object w/ name 'AnaK' AKA: ['NGC7716', 'NSA61945']> 28
<hosts.NSAHost object w/ name 'Dune' AKA: ['NGC5750', 'NSA165536']> 49
<hosts.NSAHost object w/ name 'Sopranos' AKA: ['NGC4045', 'NSA13927']> 40
<hosts.NSAHost object w/ name 'Narnia' AKA: ['NGC1015', 'NSA132339']> 28
<hosts.NSAHost object w/ name 'Bandamanna' AKA: ['NGC7818', 'NSA126115']> 32
<hosts.NSAHost object w/ name 'MobyDick' AKA: ['NGC3067', 'NSA85746']> 0
<hosts.NSAHost object w/ name 'StarTrek' AKA: ['NGC2543', 'NSA33446']> 0
<hosts.NSAHost object w/ name 'OBrother' AKA: ['NGCPGC068743', 'NSA149781']> 7
<hosts.NSAHost object w/ name 'Catch22' AKA: ['NGC7541', 'NSA150887']> 4
<hosts.NSAHost object w/ name 'HarryPotter' AKA: ['NGCPGC004948', 'NSA129237']> 25
<hosts.NSAHost object w/ name 'O

In [41]:
def load_host_catalog(hostobj, drnum):
    host_bricks = host_bricks_dct[drnum]
    tabs = []
    for brick in host_bricks[host_bricks['closest_host_name'] == hostobj.name]:
        catfn = 'decals_dr{}/catalogs/tractor-{}.fits'.format(drnum, brick['brickname'])
        tabs.append(Table.read(catfn))
    if tabs:
        return table.vstack(tabs)

In [49]:
for hostobj in tqdm.tqdm_notebook(paperhosts):
    hostobj.cats3 = load_host_catalog(hostobj, 3)
    
[(hostobj.name, len(hostobj.cats3)) for hostobj in paperhosts if hostobj.cats3 is not None]

[('Odyssey', 145041),
 ('Alice', 237279),
 ('Gilgamesh', 94874),
 ('AnaK', 182540),
 ('Dune', 231112),
 ('Sopranos', 286959),
 ('Narnia', 158462),
 ('Bandamanna', 59030),
 ('OBrother', 17901),
 ('Catch22', 3951),
 ('HarryPotter', 44918),
 ('Oz', 41667),
 ('ScoobyDoo', 94704),
 ('Othello', 138656)]

In [47]:
for hostobj in tqdm.tqdm_notebook(paperhosts):
    hostobj.cats4 = cat = load_host_catalog(hostobj, 4)
[(hostobj.name, len(hostobj.cats4)) for hostobj in paperhosts if hostobj.cats4 is not None]

[('MobyDick', 86799), ('StarTrek', 86644)]

In [ ]:
for hostobj in tqdm.tqdm_notebook(paperhosts):
    for cat in (hostobj.cats3, hostobj.cats4):
        if cat is not None:
            cat['objname'] = ['{}-{}'.format(row['brickname'], row['objid']) for row in cat]

In [52]:
paperhosts[0].cats3

brickid,brickname,objid,brick_primary,blob,ninblob,tycho2inblob,type,ra,ra_ivar,dec,dec_ivar,bx,by,bx0,by0,left_blob,out_of_bounds,dchisq [5],ebv,cpu_source,cpu_blob,blob_width,blob_height,blob_npix,blob_nimages,blob_totalpix,decam_flux [6],decam_flux_ivar [6],"decam_apflux [6,8]","decam_apflux_resid [6,8]","decam_apflux_ivar [6,8]",decam_mw_transmission [6],decam_nobs [6],decam_rchi2 [6],decam_fracflux [6],decam_fracmasked [6],decam_fracin [6],decam_anymask [6],decam_allmask [6],decam_psfsize [6],wise_flux [4],wise_flux_ivar [4],wise_mw_transmission [4],wise_nobs [4],wise_fracflux [4],wise_rchi2 [4],"wise_lc_flux [2,5]","wise_lc_flux_ivar [2,5]","wise_lc_nobs [2,5]","wise_lc_fracflux [2,5]","wise_lc_rchi2 [2,5]","wise_lc_mjd [2,5]",fracDev,fracDev_ivar,shapeExp_r,shapeExp_r_ivar,shapeExp_e1,shapeExp_e1_ivar,shapeExp_e2,shapeExp_e2_ivar,shapeDev_r,shapeDev_r_ivar,shapeDev_e1,shapeDev_e1_ivar,shapeDev_e2,shapeDev_e2_ivar,decam_depth [6],decam_galdepth [6]
,,,,,,,,deg,1/deg^2,deg,1/deg^2,,,,,,,,mag,,,,,,,,nanomaggy,1/nanomaggy^2,nanomaggy,nanomaggy,1/nanomaggy^2,,,,,,,,,,nanomaggy,1/nanomaggy^2,,,,,nanomaggy,1/nanomaggy^2,,,,,,,arcsec,1/arcsec^2,,,,,arcsec,1/arcsec^2,,,,,1/nanomaggy^2,1/nanomaggy^2
int32,str8,int32,bool,int32,int16,bool,str4,float64,float32,float64,float32,float32,float32,float32,float32,bool,bool,float32,float32,float32,float32,int16,int16,int32,int16,int32,float32,float32,float32,float32,float32,float32,uint8,float32,float32,float32,float32,int16,int16,float32,float32,float32,float32,int16,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
441620,2473p195,0,False,0,1,False,PSF,247.495475827,1.48138e+11,19.3702523218,1.46109e+11,142.49,17.3175,142.0,17.0,False,False,10894.8 .. 0.0,0.0523248,4.9,5.04,34,35,910,12,10689,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.824869 .. 0.950015,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,-1.14857 .. -365.795,2.03034 .. 2.42482e-05,0.991172 .. 0.999562,74 .. 14,1.3144 .. 0.842091,0.75935 .. 0.102832,0.0828545 .. 0.0,0.388915 .. 0.0,14 .. 0,16.5635 .. 0.0,0.344379 .. 0.0,55248.1 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
441620,2473p195,1,False,1,2,False,PSF,247.470297457,7.70418e+08,19.3694017375,7.80088e+08,468.864,5.41228,469.0,6.0,False,False,87.7037 .. 0.0,0.0523094,1.33,6.16,43,19,542,11,5785,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.824916 .. 0.95003,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,-0.522741 .. 135.125,2.07265 .. 2.11677e-05,0.991174 .. 0.999562,74 .. 14,1.08924 .. 2.27861,0.356939 .. 0.168483,-4.39709 .. 0.0,0.3881 .. 0.0,14 .. 0,0.222421 .. 0.0,0.773632 .. 0.0,55248.1 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
441620,2473p195,2,False,1,2,False,PSF,247.468970182,1.18565e+09,19.3696753758,1.04249e+09,486.072,9.16206,486.0,9.0,False,False,42.454 .. 0.0,0.0523086,0.83,6.16,43,19,542,11,5785,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.824918 .. 0.95003,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,-1.16562 .. -277.789,2.12133 .. 2.26824e-05,0.991174 .. 0.999562,74 .. 14,0.292451 .. 0.696415,0.744256 .. 0.165091,1.01786 .. 0.0,0.394927 .. 0.0,14 .. 0,1.8935 .. 0.0,0.521322 .. 0.0,55248.1 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 .. 0.0,0.0 .. 0.0
441620,2473p195,3,False,2,4,False,PSF,247.46302259,4.75784e+13,19.3716610909,4.95511e+13,563.184,36.403,563.0,36.0,False,False,3.11851e+06 .. 3.11898e+06,0.0523033,9.87,15.01,69,74,3192,12,36650,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.824934 .. 0.950035,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,38.9347 .. 221.097,2.07125 .. 2.52506e-05,0.991175 .. 0.999562,74 .. 14,0.0120547 .. 0.682399,0.846584